## Imports

In [ ]:
import sys 
import numpy as np 
import matplotlib.pyplot as plt


import torch
import torchvision 
import torch.optim as optim
import torch.nn as nn


print("Python: %s" % sys.version)
print("Pytorch: %s" % torch.__version__)

## Hyperparameters

In [ ]:
n_epochs = 10
batch_size = 4 
lr = 0.001

## Load CIFAR10

In [ ]:
# define series of transforms to pre process images 
transform = torchvision.transforms.Compose([
    torchvision.transforms.Pad(2),
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

classes = ["airplane", "automobile", "bird", "cat", "deer", "dog", "frog", "horse", "ship", "truck"]


# load training set 
cifar10_trainset = torchvision.datasets.CIFAR10('datasets/', train=True, transform=transform, download=True)
cifar10_trainloader = torch.utils.data.DataLoader(cifar10_trainset, batch_size=batch_size, shuffle=True, num_workers=2)

# load test set 
cifar10_testset = torchvision.datasets.CIFAR10('datasets/', train=False, transform=transform, download=True)
cifar10_testloader = torch.utils.data.DataLoader(cifar10_testset, batch_size=1, shuffle=False, num_workers=2)

# helper function to unnormalize and plot image 
def imshow(img):
    img = np.array(img)
    img = img / 2 + 0.5
    img = np.moveaxis(img, 0, -1)
    plt.imshow(img)
    
# display sample from dataset 
imgs,labels = iter(cifar10_trainloader).next()
imshow(torchvision.utils.make_grid(imgs))  

## Model Architecture

In [ ]:
# determine device to run network on (runs on gpu if available)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# load the torchvision resnet18 implementation 
resnet18 = torchvision.models.resnet18(num_classes=10)
# resnet fix for cifar10 image size 
resnet18.avgpool = nn.AvgPool2d(2, stride=1)

#alexnet = torchvision.models.alexnet()
#vgg16 = torchvision.models.vgg16()
#squeezenet = torchvision.models.squeezenet1_0()
#densenet = torchvision.models.densenet161()
#inception = torchvision.models.inception_v3()

net = resnet18.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=lr, momentum=0.9)

## Training 

In [ ]:
def eval_net(): 
    class_correct = np.zeros(10)
    class_total = np.zeros(10)
    
    with torch.no_grad():
        for data in cifar10_testloader:
            img, lbl = data
            img, lbl = img.to(device), lbl.to(device)
            
            output = net(img)
            predicted = torch.argmax(output)
            correct = predicted == lbl

            
            class_correct[lbl] += correct
            class_total[lbl] += 1 
            
    for i in range(10):
        print('Accuracy of %s : %f %%' % (classes[i], 100 * class_correct[i] / class_total[i]))


for epoch in range(n_epochs): 
    
    for i, batch in enumerate(cifar10_trainloader): 
        
        imgs, labels = batch
        imgs, labels = imgs.to(device), labels.to(device)

        
        optimizer.zero_grad()
        
        outputs = net(imgs)
        
        
        loss = criterion(outputs, labels)
        loss.backward()
        
        optimizer.step()
        
        print("[%d/%d][%d/%d] loss = %f" % (epoch, n_epochs, i, len(cifar10_trainloader), loss.item()))

    # evaluate performance on testset at the end of each epoch 
    eval_net()
        
